In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Bidirectional, LSTM, Reshape, RepeatVector, TimeDistributed
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
def load_data():
        x_train = np.load(r'C:\Users\Vee\Desktop\python\answers.npy',allow_pickle=True)
        x_train = x_train.reshape(721,4,4)
        return x_train

In [40]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 4
        self.img_cols = 4
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols)
        self.latent_dim = 16

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(4,4))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(4, 4)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        #specifying output to have 40 timesteps
        model.add(RepeatVector(16))
        #specifying 1 feature as the output
        model.add(Bidirectional(LSTM(64, activation = 'selu', return_sequences=True, dropout = 0.2)))
        model.add(Bidirectional(LSTM(64, activation = 'relu', return_sequences=True, dropout = 0.2)))
        model.add(Bidirectional(LSTM(64, activation = 'tanh', return_sequences=True, dropout = 0.2)))
        model.add(TimeDistributed(Dense(64, activation = 'selu')))
        model.add(Dropout(0.3))
        model.add(TimeDistributed(Dense(64, activation = 'selu')))
        model.add(Dropout(0.3))
        model.add(TimeDistributed(Dense(64, activation = 'selu')))
        model.add(Dropout(0.3))
        model.add(TimeDistributed(Dense(1, activation = 'linear')))
        model.summary()

        noise = Input(shape=(4,4))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Bidirectional(LSTM(64, activation = 'tanh', dropout = 0.2), input_shape=(16,1)))
        model.add(RepeatVector(1))
        model.add(TimeDistributed(Dense(64, activation = 'selu')))
        model.add(Dropout(0.3))
        model.add(TimeDistributed(Dense(64, activation = 'selu')))
        model.add(Dropout(0.3))
        model.add(TimeDistributed(Dense(64, activation = 'selu')))
        model.add(Dropout(0.3))
        model.add(TimeDistributed(Dense(1, activation = 'linear')))


        model.summary()

        return model
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size,1,1))
        fake = np.zeros((batch_size,1,1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            imgs = np.array(imgs)
            imgs = imgs.reshape(len(imgs),16,1)

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size,4,4))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                #self.save_imgs(epoch)
                self.generator.save("generator.h5")

In [ ]:
dcgan = DCGAN()
dcgan.train(epochs=1001, batch_size=10, save_interval=100)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_83 (Bidirectio (None, 128)               33792     
_________________________________________________________________
repeat_vector_20 (RepeatVect (None, 1, 128)            0         
_________________________________________________________________
time_distributed_109 (TimeDi (None, 1, 64)             8256      
_________________________________________________________________
dropout_82 (Dropout)         (None, 1, 64)             0         
_________________________________________________________________
time_distributed_110 (TimeDi (None, 1, 64)             4160      
_________________________________________________________________
dropout_83 (Dropout)         (None, 1, 64)             0         
_________________________________________________________________
time_distributed_111 (TimeDi (None, 1, 64)           

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 4.227586, acc.: 50.00%] [G loss: 5.144736]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.983242, acc.: 50.00%] [G loss: 6.013484]
2 [D loss: 1.696669, acc.: 50.00%] [G loss: 3.178210]
3 [D loss: 1.593351, acc.: 50.00%] [G loss: 5.966186]
4 [D loss: 1.532439, acc.: 50.00%] [G loss: 3.390188]
5 [D loss: 1.623617, acc.: 50.00%] [G loss: 3.210495]
6 [D loss: 1.568402, acc.: 50.00%] [G loss: 2.897941]
7 [D loss: 1.428355, acc.: 50.00%] [G loss: 2.666708]
8 [D loss: 1.431103, acc.: 50.00%] [G loss: 2.685489]
9 [D loss: 1.397308, acc.: 50.00%] [G loss: 2.567030]
10 [D loss: 1.386629, acc.: 50.00%] [G loss: 3.063162]
11 [D loss: 1.380127, acc.: 50.00%] [G loss: 2.629270]
12 [D loss: 1.346286, acc.: 50.00%] [G loss: 2.432515]
13 [D loss: 1.356521, acc.: 50.00%] [G loss: 2.401710]
14 [D loss: 1.204844, acc.: 50.00%] [G loss: 2.284453]
15 [D loss: 1.185509, acc.: 50.00%] [G loss: 2.176377]
16 [D loss: 1.241452, acc.: 50.00%] [G loss: 1.965274]
17 [D loss: 1.255606, acc.: 50.00%] [G loss: 2.217109]
18 [D loss: 1.103330, acc.: 50.00%] [G loss: 1.969199]
19 [D loss: 1.05683

150 [D loss: 0.728783, acc.: 60.00%] [G loss: 0.707366]
151 [D loss: 0.666070, acc.: 70.00%] [G loss: 0.812464]
152 [D loss: 0.735646, acc.: 45.00%] [G loss: 0.809903]
153 [D loss: 0.773793, acc.: 40.00%] [G loss: 0.886046]
154 [D loss: 0.750533, acc.: 50.00%] [G loss: 0.790004]
155 [D loss: 0.826161, acc.: 30.00%] [G loss: 0.738916]
156 [D loss: 0.744229, acc.: 55.00%] [G loss: 0.937043]
157 [D loss: 0.692639, acc.: 50.00%] [G loss: 0.749999]
158 [D loss: 0.856111, acc.: 35.00%] [G loss: 0.703155]
159 [D loss: 0.780204, acc.: 45.00%] [G loss: 0.726770]
160 [D loss: 0.763089, acc.: 40.00%] [G loss: 0.755306]
161 [D loss: 0.757342, acc.: 55.00%] [G loss: 0.781765]
162 [D loss: 0.656693, acc.: 70.00%] [G loss: 0.754333]
163 [D loss: 0.743404, acc.: 55.00%] [G loss: 0.836640]
164 [D loss: 0.731319, acc.: 55.00%] [G loss: 0.738024]
165 [D loss: 0.766353, acc.: 40.00%] [G loss: 0.796059]
166 [D loss: 0.816363, acc.: 45.00%] [G loss: 0.905594]
167 [D loss: 0.698586, acc.: 60.00%] [G loss: 0.

297 [D loss: 0.620681, acc.: 55.00%] [G loss: 0.744812]
298 [D loss: 0.723685, acc.: 50.00%] [G loss: 0.680972]
299 [D loss: 0.802286, acc.: 25.00%] [G loss: 0.895308]
300 [D loss: 0.746641, acc.: 40.00%] [G loss: 0.798339]
301 [D loss: 0.734519, acc.: 45.00%] [G loss: 0.783483]
302 [D loss: 0.756972, acc.: 35.00%] [G loss: 0.752837]
303 [D loss: 0.838490, acc.: 25.00%] [G loss: 0.886296]
304 [D loss: 0.769305, acc.: 35.00%] [G loss: 0.790398]
305 [D loss: 0.676940, acc.: 55.00%] [G loss: 0.924132]
306 [D loss: 0.694638, acc.: 60.00%] [G loss: 0.715605]
307 [D loss: 0.709489, acc.: 40.00%] [G loss: 0.797656]
308 [D loss: 0.720628, acc.: 55.00%] [G loss: 0.844555]
309 [D loss: 0.649452, acc.: 55.00%] [G loss: 0.686077]
310 [D loss: 0.703416, acc.: 40.00%] [G loss: 0.904353]
311 [D loss: 0.710205, acc.: 55.00%] [G loss: 0.708382]
312 [D loss: 0.678206, acc.: 55.00%] [G loss: 0.863158]
313 [D loss: 0.818598, acc.: 35.00%] [G loss: 0.768960]
314 [D loss: 0.718520, acc.: 35.00%] [G loss: 0.

445 [D loss: 0.753498, acc.: 40.00%] [G loss: 0.899105]
446 [D loss: 0.630685, acc.: 60.00%] [G loss: 0.734485]
447 [D loss: 0.730197, acc.: 45.00%] [G loss: 0.795581]
448 [D loss: 0.814045, acc.: 20.00%] [G loss: 0.755053]
449 [D loss: 0.693827, acc.: 75.00%] [G loss: 0.758146]
450 [D loss: 0.737154, acc.: 45.00%] [G loss: 0.681597]
451 [D loss: 0.730821, acc.: 50.00%] [G loss: 0.860090]
452 [D loss: 0.774155, acc.: 50.00%] [G loss: 0.836411]
453 [D loss: 0.842172, acc.: 30.00%] [G loss: 0.858687]
454 [D loss: 0.740178, acc.: 40.00%] [G loss: 0.842146]
455 [D loss: 0.720369, acc.: 45.00%] [G loss: 0.667479]
456 [D loss: 0.732263, acc.: 45.00%] [G loss: 0.757342]
457 [D loss: 0.788387, acc.: 45.00%] [G loss: 0.827439]
458 [D loss: 0.740738, acc.: 40.00%] [G loss: 0.756041]
459 [D loss: 0.701515, acc.: 55.00%] [G loss: 0.721777]
460 [D loss: 0.705875, acc.: 50.00%] [G loss: 0.942217]
461 [D loss: 0.772336, acc.: 30.00%] [G loss: 0.808519]
462 [D loss: 0.694974, acc.: 55.00%] [G loss: 0.

592 [D loss: 0.787607, acc.: 30.00%] [G loss: 0.829971]
593 [D loss: 0.746423, acc.: 45.00%] [G loss: 0.713573]
594 [D loss: 0.679537, acc.: 60.00%] [G loss: 0.697024]
595 [D loss: 0.709468, acc.: 45.00%] [G loss: 0.796578]
596 [D loss: 0.652169, acc.: 60.00%] [G loss: 0.717496]
597 [D loss: 0.729458, acc.: 50.00%] [G loss: 0.734680]
598 [D loss: 0.751603, acc.: 35.00%] [G loss: 0.736377]
599 [D loss: 0.726689, acc.: 45.00%] [G loss: 0.735143]
600 [D loss: 0.673048, acc.: 55.00%] [G loss: 0.649779]
601 [D loss: 0.687266, acc.: 60.00%] [G loss: 0.837164]
602 [D loss: 0.732196, acc.: 55.00%] [G loss: 1.005815]
603 [D loss: 0.845770, acc.: 35.00%] [G loss: 0.747662]
604 [D loss: 0.689804, acc.: 50.00%] [G loss: 0.789453]
605 [D loss: 0.716042, acc.: 50.00%] [G loss: 0.862322]
606 [D loss: 0.755195, acc.: 45.00%] [G loss: 0.904441]
607 [D loss: 0.657265, acc.: 50.00%] [G loss: 0.870365]
608 [D loss: 0.683324, acc.: 55.00%] [G loss: 0.836252]
609 [D loss: 0.679786, acc.: 60.00%] [G loss: 0.